To use Padé approximants for approximating sin (x), first find the Taylor series of sin (x) around x=0, then equate it to a rational function P(x)/Q(x) where P(x) and Q(x) are polynomials with Q(x)'s constant term being 1

In [1]:
import numpy as np
from scipy.interpolate import pade
import matplotlib.pyplot as plt